# Preprocessing: LL14


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'LL14'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/LL14/ieeg/LL14_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/LL14/ieeg/LL14_raw_clean_ieeg.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.


In [4]:
## Channels did not include probe prefixes in bci data file, convert to descriptive names here

# read in csv with correct names
channel_csv = pd.read_csv(f"{preproc_data_dir}/../LL14_channel_names.csv")

# convert to dict
channel_dict = dict(zip(channel_csv.bci_names, channel_csv.use_names))

# rename
mne.rename_channels(filtered_clean_fif.info, channel_dict)
mne.rename_channels(raw_clean_fif.info, channel_dict)

# add missing badd channels
filtered_clean_fif.info['bads'].extend(['CH122', 'CH123'])
raw_clean_fif.info['bads'].extend(['CH122', 'CH123'])

## Bipolar Rereferencing

In [5]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [6]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

LA1 noref
LA2 LA3
LA3 LA4
LA4 LA5
LA5 LA6
LA6 LA7
LA8 noref
LH1 LH2
LH2 LH3
LH3 LH4
LH4 LH5
LH5 LH6
LH6 LH7
LH7 LH8
LHP1 LHP2
LHP2 LHP3
LHP3 LHP4
LHP4 LHP5
LHP5 LHP6
LHP6 LHP7
LHP7 LHP8
LOF1 LOF2
LOF2 LOF3
LOF3 LOF4
LOF4 LOF5
LOF5 LOF6
LOF6 LOF7
LOF7 LOF8
LOF8 LOF9
LOF9 LOF10
LOF10 LOF11
LOF11 LOF12
LAC1 LAC2
LAC2 LAC3
LAC3 LAC4
LAC4 LAC5
LAC5 LAC6
LAC6 LAC7
LAC7 LAC8
LAC8 LAC9
LAC9 LAC10
LPC1 LPC2
LPC2 LPC3
LPC3 LPC4
LPC4 LPC5
LPC5 LPC6
LPC6 LPC7
LPC7 LPC8
LPC8 LPC9
LPC9 LPC10
RA1 RA2
RA2 RA3
RA3 RA4
RA4 RA5
RA5 RA6
RA6 RA7
RA7 RA8
RH1 noref
RH2 RH3
RH3 RH4
RH4 RH5
RH5 RH6
RH6 RH7
RH7 RH8
RHP1 RHP2
RHP2 RHP3
RHP3 RHP4
RHP4 RHP5
RHP5 RHP6
RHP6 RHP7
RHP7 RHP8
ROF1 ROF2
ROF2 ROF3
ROF3 ROF4
ROF4 ROF5
ROF5 ROF6
ROF6 ROF7
ROF7 ROF8
ROF8 ROF9
ROF9 ROF10
ROF10 ROF11
ROF11 ROF12
RAC1 RAC2
RAC2 RAC3
RAC3 RAC4
RAC4 RAC5
RAC5 RAC6
RAC6 RAC7
RAC7 RAC8
RAC8 RAC9
RAC9 RAC10
RMC1 RMC2
RMC2 RMC3
RMC3 RMC4
RMC4 RMC5
RMC5 RMC6
RMC6 RMC7
RMC7 RMC8
RPC1 RPC2
RPC2 RPC3
RPC3 RPC4
RPC4 RPC5
RPC5 RPC6
RPC6 RP

In [7]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 2344719  =      0.000 ...  4579.529 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=106, n_times=2344720
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.
Added the following bipolar channels:
LA2-LA3, LA3-LA4, LA4-LA5, LA5-LA6, LA6-LA7, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LH6-LH7, LH7-LH8, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LHP6-LHP7, LHP7-LHP8, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LOF10-LOF11, LOF11-LOF12, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, LAC8-LAC9, LAC9-LAC10, LPC1-LPC2, LPC2-LPC3, LPC3-LPC4, LPC4-LPC5, LPC5-LPC6, LPC6-LPC7, LPC7-LPC8, LPC8-LPC9, LPC9-LPC10, RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RA6-RA7, RA7-RA8, RH2-RH3, RH3-RH4, RH4-RH5, RH5-RH6, RH6-RH7, RH7-RH8, RHP1-RHP2, RHP2-RHP3, RHP3-RHP4, RHP4-RHP5, RHP5-RHP6, RHP6-RHP7, RHP7-RHP8, ROF1-ROF2, ROF2-ROF

/tmp/ipykernel_78456/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif
[done]
Reading 0 ... 2344719  =      0.000 ...  4579.529 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=106, n_times=2344720
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.
Added the following bipolar channels:
LA2-LA3, LA3-LA4, LA4-LA5, LA5-LA6, LA6-LA7, LH1-LH2, LH2-LH3, LH3-LH4, LH4-LH5, LH5-LH6, LH6-LH7, LH7-LH8, LHP1-LHP2, LHP2-LHP3, LHP3-LHP4, LHP4-LHP5, LHP5-LHP6, LHP6-LHP7, LHP7-LHP8, LOF1-LOF2, LOF2-LOF3, LOF3-LOF4, LOF4-LOF5, LOF5-LOF6, LOF6-LOF7, LOF7-LOF8, LOF8-LOF9, LOF9-LOF10, LOF10-LOF11, LOF11-LOF12, LAC1-LAC2, LAC2-LAC3, LAC3-LAC4, LAC4-LAC5, LAC5-LAC6, LAC6-LAC7, LAC7-LAC8, LAC8-LAC9, LAC9-LAC10, LPC1-LPC2, LPC2-LPC3, LPC3-LPC4, LPC4-LPC5, LPC5-LPC6, LPC6-LPC7, LPC7-LPC8, LPC8-LPC9, LPC9-LPC10, RA1-RA2, RA2-RA3, RA3-RA4, RA4-RA5, RA5-RA6, RA6-RA7, RA7-RA8, RH2-RH3, RH3-RH4, RH4-RH5, RH5-RH6, RH6-RH7, RH7-RH8, RHP1-

/tmp/ipykernel_78456/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Closing /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_raw_clean_data.fif
[done]


In [8]:
# Visualize it #
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')
bp_filtered_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 160, clipping = None, event_color = 'r')

Opening raw data file /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.


/tmp/ipykernel_5031/3331281758.py:3: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


1642 events found
Event IDs: [0 1]
Using qt as 2D backend.


/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/scipy/stats/_stats_py.py:3146: RuntimeWarning: invalid value encountered in subtract
  z = (scores - mn) / std
/home/brooke/mambaforge/envs/ieeg_analysis2/lib/python3.9/site-packages/mne_qt_browser/_pg_figure.py:4338: RuntimeWarning: invalid value encountered in subtract
  self.mne.data = self.mne.data - np.nanmean(


## Epoching the data 

### Onset

In [12]:
# # read in csv with correct names
# trial_times_df = pd.read_csv(f"{raw_data_dir}/behave/LL14_trial_times.csv")

# # convert to dict
# trial_times = dict(zip(trial_times_df.neural_trial_numeric, trial_times_df.trial_length))
trial_times

{0: 6.900390625,
 1: 8.900390625,
 2: 1.849609375,
 3: 6.650390625,
 4: 3.75,
 5: 0.400390625,
 6: 3.5,
 7: 4.25,
 8: 3.548828125,
 9: 3.8984375,
 10: 5.599609375,
 11: 3.099609375,
 12: 7.75,
 13: 3.75,
 14: 2.0,
 15: 5.298828125,
 16: 7.150390625,
 17: 4.6015625,
 18: 5.298828125,
 19: 4.599609375,
 20: 4.25,
 21: 3.349609375,
 22: 4.548828125,
 23: 4.3515625,
 24: 0.650390625,
 25: 5.298828125,
 26: 5.599609375,
 27: 4.55078125,
 28: 4.25,
 29: 5.44921875,
 30: 4.048828125,
 31: 5.900390625,
 32: 1.951171875,
 33: 6.349609375,
 34: 0.650390625,
 35: 3.80078125,
 36: 4.099609375,
 37: 4.5,
 38: 6.69921875,
 39: 5.75,
 40: 3.951171875,
 41: 4.650390625,
 42: 4.55078125,
 43: 4.94921875,
 44: 3.6015625,
 45: 3.69921875,
 46: 5.650390625,
 47: 0.701171875,
 48: 4.30078125,
 49: 4.548828125,
 50: 3.900390625,
 51: 4.5,
 52: 4.25,
 53: 4.94921875,
 54: 4.349609375,
 55: 3.599609375,
 56: 7.150390625,
 57: 4.400390625,
 58: 3.44921875,
 59: 4.1484375,
 60: 4.25,
 61: 4.150390625,
 62: 3.15

In [8]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events[0:-4], 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.


/tmp/ipykernel_78456/1818622122.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


1642 events found
Event IDs: [0 1]
Not setting metadata
819 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 8193 original time points ...
Splitting into 2 parts
Loading data for 410 events and 8193 original time points ...


/tmp/ipykernel_78456/1818622122.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 409 events and 8193 original time points ...


### Trial End

In [9]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-4]

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.


/tmp/ipykernel_78456/2945425420.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


1642 events found
Event IDs: [0 1]
Not setting metadata
819 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 819 events and 5121 original time points ...


/tmp/ipykernel_78456/2945425420.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


### Last Away

In [10]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,1,4,12774,0,1
1,2,8,15308,0,1
2,3,20,17996,0,1
3,6,2,25139,0,1
4,7,5,27852,0,1
...,...,...,...,...,...
721,815,4,2329318,0,1
722,816,13,2332313,0,1
723,817,16,2335385,0,1
724,818,10,2337459,0,1


In [11]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [12]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.
Not setting metadata
726 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 726 events and 5121 original time points ...


/tmp/ipykernel_78456/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78456/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


### First Dot

In [13]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,15,7654,0,1
1,1,4,11315,0,1
2,3,20,17382,0,1
3,4,16,20352,0,1
4,6,2,24755,0,1
...,...,...,...,...,...
775,815,4,2328755,0,1
776,816,13,2331904,0,1
777,817,16,2334950,0,1
778,818,10,2336972,0,1


In [14]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [15]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.
Not setting metadata
780 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 4609 original time points ...


/tmp/ipykernel_78456/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78456/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 780 events and 4609 original time points ...


## First Move

In [16]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,15,7526,0,1
1,1,4,9984,0,1
2,2,8,15308,0,1
3,3,20,17356,0,1
4,4,16,20172,0,1
...,...,...,...,...,...
810,815,4,2328729,0,1
811,816,13,2331673,0,1
812,817,16,2334617,0,1
813,818,10,2336844,0,1


In [17]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [18]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.
Not setting metadata
815 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 815 events and 5121 original time points ...


/tmp/ipykernel_78456/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78456/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


## Ghost Attack

In [19]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,1,4,12569,0,1
1,4,16,21427,0,1
2,12,7,43033,0,1
3,26,2,83379,0,1
4,30,10,94848,0,1
...,...,...,...,...,...
164,807,3,2304768,0,1
165,809,13,2309632,0,1
166,810,11,2315008,0,1
167,812,9,2321356,0,1


In [20]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [21]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif...
    Range : 0 ... 2344719 =      0.000 ...  4579.529 secs
Ready.
Not setting metadata
169 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 5121 original time points ...
Loading data for 169 events and 5121 original time points ...


/tmp/ipykernel_78456/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_78456/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/LL14/ieeg/LL14_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
